This example demonstrates bundle adjustment of camera extrinsics and intrinsics, as well as 3D landmark positions, for a Structure-from-Motion problem. The example isn't particularly optimized for performance, but demonstrates the simplest way to set this up with SymForce.

We use the Bundle-Adjustment-in-the-Large dataset, as described here: https://grail.cs.washington.edu/projects/bal/

Feature correspondences have already been selected, and we're given initial guesses for all of the variables; our only task is to perform bundle adjustment.

The camera model is a simple polynomial model, and each image is assumed to be captured by a different camera with its own intrinsics.

Ceres and GTSAM also have reference implementations for this dataset, see [here](https://github.com/ceres-solver/ceres-solver/blob/master/examples/simple_bundle_adjuster.cc) for Ceres and [here](https://github.com/devbharat/gtsam/blob/master/examples/SFMExample_bal.cpp) for GTSAM.

In [5]:
"""
Script to download Bundle-Adjustment-in-the-Large datasets into the `./data` folder
"""

import bz2
from pathlib import Path

import requests

# CURRENT_DIR = Path(__file__).resolve().parent

URL = "https://grail.cs.washington.edu/projects/bal/data/trafalgar/"

PROBLEMS = [
    "problem-21-11315-pre.txt",
    "problem-39-18060-pre.txt",
    "problem-50-20431-pre.txt",
    "problem-126-40037-pre.txt",
    "problem-138-44033-pre.txt",
    "problem-161-48126-pre.txt",
    "problem-170-49267-pre.txt",
    "problem-174-50489-pre.txt",
    "problem-193-53101-pre.txt",
    "problem-201-54427-pre.txt",
    "problem-206-54562-pre.txt",
    "problem-215-55910-pre.txt",
    "problem-225-57665-pre.txt",
    "problem-257-65132-pre.txt",
]


Path("data").mkdir(exist_ok=True)
for problem in PROBLEMS:
    result = requests.get(URL + problem + ".bz2")
    result_uncompressed = bz2.decompress(result.content)
    (Path("data") / problem).write_bytes(result_uncompressed)

In [6]:
"""
Symbolic factor and codegen for the Bundle-Adjustment-in-the-Large problem
"""

from pathlib import Path

import symforce.symbolic as sf
from symforce import codegen
from symforce.codegen import values_codegen
from symforce.values import Values


def snavely_reprojection_residual(
    cam_T_world: sf.Pose3,
    intrinsics: sf.V3,
    point: sf.V3,
    pixel: sf.V2,
    epsilon: sf.Scalar,
) -> sf.V2:
    """
    Reprojection residual for the camera model used in the Bundle-Adjustment-in-the-Large dataset, a
    polynomial camera with two distortion coefficients, cx == cy == 0, and fx == fy

    See https://grail.cs.washington.edu/projects/bal/ for more information

    Args:
        cam_T_world: The (inverse) pose of the camera
        intrinsics: Camera intrinsics (f, k1, k2)
        point: The world point to be projected
        pixel: The measured pixel in the camera (with (0, 0) == center of image)

    Returns:
        residual: The reprojection residual
    """
    focal_length, k1, k2 = intrinsics

    # Here we're writing the projection ourselves because this isn't a camera model provided by
    # SymForce.  For cameras in `symforce.cam` we could just create a `sf.PosedCamera` and call
    # `camera.pixel_from_global_point` instead, or we could create a subclass of `sf.CameraCal` and
    # do that.
    point_cam = cam_T_world * point

    p = sf.V2(point_cam[:2]) / sf.Max(-point_cam[2], epsilon)

    r = 1 + k1 * p.squared_norm() + k2 * p.squared_norm() ** 2

    pixel_projected = focal_length * r * p

    return pixel_projected - pixel


def generate(output_dir: Path) -> None:
    """
    Generates the snavely_reprojection_factor into C++, as well as a set of Keys to help construct
    the optimization problem in C++, and puts them into `output_dir`.  This is called by
    `symforce/test/symforce_examples_bundle_adjustment_in_the_large_codegen_test.py` to generate the
    contents of the `gen` folder inside this directory.
    """

    # Generate the residual function (see `gen/snavely_reprojection_factor.h`)
    codegen.Codegen.function(snavely_reprojection_residual, codegen.CppConfig()).with_linearization(
        which_args=["cam_T_world", "intrinsics", "point"]
    ).generate_function(output_dir=output_dir, skip_directory_nesting=True)

    # Make a `Values` with variables used in the C++ problem, and generate C++ Keys for them (see
    # `gen/keys.h`)
    values = Values(
        cam_T_world=sf.Pose3(),
        intrinsics=sf.V3(),
        point=sf.V3(),
        pixel=sf.V2(),
        epsilon=sf.Scalar(),
    )

    values_codegen.generate_values_keys(
        values, output_dir, config=codegen.CppConfig(), skip_directory_nesting=True
    )

In [8]:
generate("bundle_adjustment/cpp")

codegen.__init__():141 WARNING -- 
    Generating code with epsilon set to 0 - This is dangerous!  You may get NaNs, Infs,
    or numerically unstable results from calling generated functions near singularities.

    In order to safely generate code, you should set epsilon to either a symbol
    (recommended) or a small numerical value like `sf.numeric_epsilon`.  You should do
    this before importing any other code from symforce, e.g. with

        import symforce
        symforce.set_epsilon_to_symbol()

    or

        import symforce
        symforce.set_epsilon_to_number()

    For more information on use of epsilon to prevent singularities, take a look at the
    Epsilon Tutorial: https://symforce.org/tutorials/epsilon_tutorial.html

codegen.__init__():141 WARNING -- 
    Generating code with epsilon set to 0 - This is dangerous!  You may get NaNs, Infs,
    or numerically unstable results from calling generated functions near singularities.

    In order to safely generate code

Here is how you would then run the optimization in C++, including the generated files:

```cpp
#include <fstream>

#include <spdlog/spdlog.h>

#include <sym/pose3.h>
#include <symforce/opt/factor.h>
#include <symforce/opt/optimizer.h>
#include <symforce/opt/values.h>

#include "./gen/keys.h"
#include "./gen/snavely_reprojection_factor.h"

using namespace sym::Keys;

/**
 * Create a `sym::Factor` for the reprojection residual, attached to the given camera and point
 * variables.  It's also attached to fixed entries in the Values for the pixel measurement and the
 * constant EPSILON.
 */
sym::Factord MakeFactor(int camera, int point, int pixel) {
  return sym::Factord::Hessian(sym::SnavelyReprojectionFactor<double>,
                               /* all_keys = */
                               {
                                   sym::Key::WithSuper(CAM_T_WORLD, camera),
                                   sym::Key::WithSuper(INTRINSICS, camera),
                                   sym::Key::WithSuper(POINT, point),
                                   sym::Key::WithSuper(PIXEL, pixel),
                                   EPSILON,
                               },
                               /* optimized_keys = */
                               {
                                   sym::Key::WithSuper(CAM_T_WORLD, camera),
                                   sym::Key::WithSuper(INTRINSICS, camera),
                                   sym::Key::WithSuper(POINT, point),
                               });
}

/**
 * A struct to represent the problem definition
 */
struct Problem {
  std::vector<sym::Factord> factors;
  sym::Valuesd values;
  int num_cameras;
  int num_points;
  int num_observations;
};

/**
 * Read the problem description from the given path
 *
 * See https://grail.cs.washington.edu/projects/bal/ for file format description
 */
Problem ReadProblem(const std::string& filename) {
  std::ifstream file(filename);

  int num_cameras, num_points, num_observations;
  file >> num_cameras;
  file >> num_points;
  file >> num_observations;

  std::vector<sym::Factord> factors;
  sym::Valuesd values;

  for (int i = 0; i < num_observations; i++) {
    int camera, point;
    file >> camera;
    file >> point;

    double px, py;
    file >> px;
    file >> py;

    factors.push_back(MakeFactor(camera, point, i));
    values.Set(sym::Key::WithSuper(PIXEL, i), Eigen::Vector2d(px, py));
  }

  for (int i = 0; i < num_cameras; i++) {
    double rx, ry, rz, tx, ty, tz, f, k1, k2;
    file >> rx;
    file >> ry;
    file >> rz;
    file >> tx;
    file >> ty;
    file >> tz;
    file >> f;
    file >> k1;
    file >> k2;

    values.Set(sym::Key::WithSuper(CAM_T_WORLD, i),
               sym::Pose3d(sym::Rot3d::FromTangent(Eigen::Vector3d(rx, ry, rz)),
                           Eigen::Vector3d(tx, ty, tz)));
    values.Set(sym::Key::WithSuper(INTRINSICS, i), Eigen::Vector3d(f, k1, k2));
  }

  for (int i = 0; i < num_points; i++) {
    double x, y, z;
    file >> x;
    file >> y;
    file >> z;

    values.Set(sym::Key::WithSuper(POINT, i), Eigen::Vector3d(x, y, z));
  }

  values.Set(EPSILON, sym::kDefaultEpsilond);

  return {std::move(factors), std::move(values), num_cameras, num_points, num_observations};
}

/**
 * Example usage: `bundle_adjustment_in_the_large_example data/problem-21-11315-pre.txt`
 */
int main(int argc, char** argv) {
  SYM_ASSERT(argc == 2);

  // Read the problem from disk, and create the Values and factors
  const auto problem = ReadProblem(argv[1]);

  // Create a copy of the Values - we'll optimize this one in place
  sym::Valuesd optimized_values = problem.values;

  // Optimize
  auto params = sym::DefaultOptimizerParams();
  params.verbose = true;
  sym::Optimizerd optimizer{params, std::move(problem.factors)};
  const auto stats = optimizer.Optimize(optimized_values);

  spdlog::info("Finished in {} iterations", stats.iterations.size());
}
```